<a href="https://colab.research.google.com/github/ldeloe/SYDE-720/blob/main/SYDE_720_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import keras
from keras import layers
from sklearn.metrics import mean_squared_error
from skimage.measure import block_reduce
import csv

**General Use Functions**

In [ ]:
def generate_scatterplot_indices(dim):
  x = []
  y = []
  for row in range(0,dim):
    for col in range(0, dim):
      x.append(row)
      y.append(col)
  return x, y

def plot_scatter(x,y,data, title, vmin, vmax):
  fig, ax = plt.subplots()
  plt.scatter(x,y,c=data, vmin = vmin, vmax = vmax)
  cbar = plt.colorbar()
  cbar.set_label("Magnitude of Velocity Field")
  ax.axes.get_xaxis().set_ticks([])
  ax.axes.get_yaxis().set_ticks([])
  plt.title(title)
  plt.show()

# function that plots training and validation loss
def plot_history(history, val_loss):
  # plot without validation loss
	if val_loss == False:
		plt.plot(history.history['loss'], label='Train')
	# plot with validation loss
	else:
		plt.plot(history.history['loss'], label='Train')
		plt.plot(history.history['val_loss'], label='Validation')

	plt.xlabel("Number of Epochs")
	plt.ylabel("Loss")
	plt.legend()
	plt.show()

def generate_plots(data, filenames, len_train, x, y,output_filename,epochs, dim):
  ufield = data[:,:,:,0]
  vfield = data[:,:,:,1]
  wfield = data[:,:,:,2]
  num_fields = 2

  normalized_divergence = []

  for i in range(0,data.shape[0]*num_fields,num_fields):
    idx=int(i/num_fields)

    divergence = compute_divergence(data[idx,0:dim,0:dim,:])
    print("The divergence is: ", (np.abs(np.sum(divergence))/(dim**2)))
    normalized_divergence.append([(len_train+idx+1)*100, epochs, np.abs(np.sum(divergence))/(dim**2)])

    plot_scatter(x,y,ufield[idx,:,:], 'U Velocity Field', -2.75, 2.75)
    plot_scatter(x,y,vfield[idx,:,:], 'V Velocity Field', -2.75, 2.75)
    plot_scatter(x,y,wfield[idx,:,:], 'W Velocity Field ',-2.75,2.75)

  with open(filepath + output_filename, 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(normalized_divergence)

def load_data_uv(filepath, filenames, percent, dim):
  validation_percent = 0.2
  train_fields = []
  test_fields = []
  train_fieldu = []
  train_fieldv = []
  train_fieldw = []
  test_fieldu = []
  test_fieldv = []
  test_fieldw = []

  train_len = int((len(filenames)/2)*percent)

  filenames_train = filenames[0:train_len*2]
  filenames_test = filenames[train_len*2:]
  val_split = round(len(filenames_train)*validation_percent)
  filenames_validation = filenames_train[-val_split:]
  filenames_train = filenames_train[:-val_split]

  field_z = np.zeros((dim,dim))
  block = 4 # corresponds to dim 512x512 downsampled to 128x128
  for i in range(0, len(filenames),2):
    if i < train_len*2:
      train = pd.read_table(filepath + filenames[i], delimiter=',',header=None)
      field = train.to_numpy()
      field = block_reduce(field, block_size=(block, block), func=np.mean)

      train2 = pd.read_table(filepath + filenames[i+1], delimiter=',',header=None)
      field2 = train2.to_numpy()
      field2 = block_reduce(field2, block_size=(block,block), func=np.mean)

      train_fieldu.append(field[0:dim,0:dim])
      train_fieldv.append(field2[0:dim,0:dim])
      train_fieldw.append(field_z)

    else:
      test = pd.read_table(filepath + filenames[i], delimiter=',',header=None)
      field = test.to_numpy()
      field = block_reduce(field, block_size=(block, block), func=np.mean)

      test2 = pd.read_table(filepath + filenames[i+1], delimiter=',',header=None)
      field2 = test2.to_numpy()
      field2 = block_reduce(field2, block_size=(block, block), func=np.mean)

      test_fieldu.append(field[0:dim,0:dim])
      test_fieldv.append(field2[0:dim,0:dim])
      test_fieldw.append(field_z)

  train = np.zeros((np.asarray(train_fieldu).shape[0],dim,dim,3))
  train[:,:,:,0] = train_fieldu
  train[:,:,:,1] = train_fieldv
  train[:,:,:,2] = train_fieldw

  test = np.zeros((np.asarray(test_fieldu).shape[0],dim,dim,3))
  test[:,:,:,0] = test_fieldu
  test[:,:,:,1] = test_fieldv
  test[:,:,:,2] = test_fieldw

  validation = train[-val_split:]
  train = train[:-val_split]

  return train, test, validation, filenames_train, filenames_test, filenames_validation

def compute_divergence(fields):
  du_dx, dv_dy, dw_dz = np.gradient(fields)
  divergence = du_dx + dv_dy + dw_dz

  return divergence

def RMSE(train_prediction, test_prediction, train, test):

  train_data = train.reshape(train.shape[0], np.prod(train.shape[1:]))
  train_pred = train_prediction.reshape(train_prediction.shape[0], np.prod(train_prediction.shape[1:]))
  test_data = test.reshape(test.shape[0], np.prod(test.shape[1:]))
  test_pred = test_prediction.reshape(test_prediction.shape[0], np.prod(test_prediction.shape[1:]))

  trainScore = np.sqrt(mean_squared_error(train_data, train_pred))
  print('Training RMSE Score: %.2f' % (trainScore))
  testScore = np.sqrt(mean_squared_error(test_data, test_pred))
  print('Test RMSE Score: %.2f' % (testScore))

def plot_mean_divergence(filenames, input_divergence,epochs,labels):

  fig, ax = plt.subplots(figsize=(16, 4))
  for i in range(0, len(filenames)):
    print(filenames[i])
    df = pd.read_csv(filepath + filenames[i], header=None)
    mean_div = []
    for j in epochs:
      divergence_at_j = df.loc[df[1] == j]
      mean_div.append(sum(divergence_at_j.iloc[:,2])/len(divergence_at_j.iloc[:,2]))

    ax.plot(epochs, mean_div, label=labels[i],linestyle='dashed',marker='o')

    print("Mean divergence at min divergence timestep for 1100: ", mean_div)
    min_idx = mean_div.index(min(mean_div))
    print("Minimum divergence is at epoch ", epochs[min_idx])

  ax.hlines(input_divergence[2], xmin=epochs[0], xmax=epochs[-1], label="Input field", color ='k') #, linewidth=2, color='r')

  plt.title("Total Absolute Divergence for Varied Epochs" )
  plt.xlabel("Number of Epochs")
  plt.ylabel(r'$\sum |\nabla \cdot V| [1/s]$')
  plt.legend()
  plt.show()

In [ ]:
tf.random.set_seed(7)

filepath = "/content/drive/MyDrive/2D-Turbulence/"
filenames = ["fileu_100.dat","filev_100.dat",
             "fileu_200.dat","filev_200.dat",
             "fileu_300.dat","filev_300.dat",
             "fileu_400.dat","filev_400.dat",
             "fileu_500.dat","filev_500.dat",
             "fileu_600.dat","filev_600.dat",
             "fileu_700.dat","filev_700.dat",
             "fileu_800.dat","filev_800.dat",
             "fileu_900.dat","filev_900.dat",
             "fileu_1000.dat","filev_1000.dat",
             "fileu_1100.dat","filev_1100.dat",
             "fileu_1200.dat","filev_1200.dat",
             "fileu_1300.dat","filev_1300.dat",
             "fileu_1400.dat","filev_1400.dat",
             "fileu_1500.dat","filev_1500.dat"]

**Plot the Original Fields for the Test Dataset without Padding**

In [ ]:
dimensions = 128
epochs = 1

x, y = generate_scatterplot_indices(dimensions)

# 70% training and validation
train_percent = 0.7
output_filename = "input_divergence.csv"

train, test, validation, filenames_train, filenames_test, filenames_validation = load_data_uv(filepath, filenames, train_percent, dimensions)
generate_plots(test, filenames_test, train.shape[0]+validation.shape[0], x,y,output_filename, epochs, dimensions)

# 35% training and validation
train_percent = 0.35
output_filename = "input_divergence_5_train.csv"

train, test, validation, filenames_train, filenames_test, filenames_validation = load_data_uv(filepath, filenames, train_percent, dimensions)
generate_plots(test, filenames_test, train.shape[0]+validation.shape[0], x,y,output_filename, epochs,dimensions)

**CAE Model**

In [ ]:
def cae_model(train, test, validation, epochs, batch_size, filenames, output_filename,dim):
  input = layers.Input(shape=(train.shape[1],train.shape[2], train.shape[3]))

  ### encoder ###
  encoder_layer1 = layers.Conv2D(6, 3, 2, activation='relu', padding='same')(input)
  encoder_layer2 = layers.Conv2D(6,  3, 2, activation='relu', padding='same')(encoder_layer1)
  encoder_layer3 = layers.Conv2D(6, 3, 2, activation='relu', padding='same')(encoder_layer2)

  ### decoder ###
  decoder_layer1 = layers.Conv2DTranspose(6, 3, strides=2, activation="relu", padding="same")(encoder_layer3)
  decoder_layer2 = layers.Conv2DTranspose(6, 3, strides=2, activation="relu", padding="same")(decoder_layer1)
  decoder_layer3 = layers.Conv2DTranspose(3, 4, strides = 2, activation=None, padding="same")(decoder_layer2)

  autoencoder = keras.Model(input, decoder_layer3)

  autoencoder.compile(optimizer="adam", loss= "mean_squared_error")
  autoencoder.summary()

  history = autoencoder.fit(train, train, epochs=epochs, batch_size=batch_size, validation_data=(validation, validation))
  plot_history(history,True)

  # model predictions
  train_prediction = autoencoder.predict(train)
  test_prediction = autoencoder.predict(test)

  # calculate RMSE train and test scores
  RMSE(train_prediction, test_prediction, train, test)

  # plot results
  x, y = generate_scatterplot_indices(test.shape[1])
  generate_plots(test_prediction, filenames_test, train.shape[0]+validation.shape[0], x,y,output_filename,epochs,dim)

In [ ]:
epochs = [15, 50, 100, 150, 200, 300, 400, 500, 600, 700, 800, 900, 1000,250,350,450,550,650,750,850,950,1050,1100,1150,1200,1250,1300,1350,1400,1450,1550,1600,1650,1700,1750,1800,1850,1900,1950]

### cae without zero padding ###
dimensions = 128

# 70% training and validation dataset
train_percent = 0.7
output_filename = 'cae_divergence.csv'

train, test, validation, filenames_train, filenames_test, filenames_validation = load_data_uv(filepath, filenames, train_percent, dimensions)
for i in range(0,len(epochs)):
  cae_model(train, test, validation, epochs[i], batch_size, filenames_test,output_filename,dimensions)

# 35% training and validation dataset
train_percent = 0.35
output_filename = 'cae_divergence_5_train.csv'

train, test, validation, filenames_train, filenames_test, filenames_validation = load_data_uv(filepath, filenames, train_percent, dimensions)
for i in range(0,len(epochs)):
  cae_model(train, test, validation, epochs[i], batch_size, filenames_test,output_filename,dimensions)

### cae with zero padding ###
dimensions = 127

# 70% training and validation dataset
output_filename = "cae_divergence_padded.csv"
train_percent = 0.7

train, test, validation, filenames_train, filenames_test, filenames_validation = load_data_uv(filepath, filenames, train_percent, dimensions)

# apply zero padding
padding = 1
train = np.pad(train, ((0,0),(padding,0),(0,padding),(0,0)), 'constant', constant_values=(0,0))
test = np.pad(test, ((0,0),(padding,0),(0,padding),(0,0)), 'constant', constant_values=(0,0))
validation = np.pad(validation, ((0,0),(padding,0),(0,padding),(0,0)), 'constant', constant_values=(0,0))

for i in range(0,len(epochs)):
  cae_model(train, test, validation, epochs[i], batch_size, filenames_test,output_filename,dimensions)

# 35% training and validation dataset
output_filename = "cae_divergence_padded_5_train.csv"
train_percent = 0.35

train, test, validation, filenames_train, filenames_test, filenames_validation = load_data_uv(filepath, filenames, train_percent, dimensions)

# apply zero padding
padding = 1
train = np.pad(train, ((0,0),(padding,0),(0,padding),(0,0)), 'constant', constant_values=(0,0))
test = np.pad(test, ((0,0),(padding,0),(0,padding),(0,0)), 'constant', constant_values=(0,0))
validation = np.pad(validation, ((0,0),(padding,0),(0,padding),(0,0)), 'constant', constant_values=(0,0))

for i in range(0,len(epochs)):
  cae_model(train, test, validation, epochs[i], batch_size, filenames_test,output_filename,dimensions)

**Phys CAE Model**

In [ ]:
class PeriodicBoundaryConditions(tf.keras.layers.Layer):
  def call(self, inputs):
    padded = []

    for i in range(0,inputs.shape[3]):
        col = inputs[slice(None),1:,:1,i]
        col = tf.transpose(col, perm=[1, 2, 0])

        crop_input = inputs[slice(None),1:,:-1,i]
        crop_input = tf.transpose(crop_input, perm=[1, 2, 0])
        pad_col = tf.concat([crop_input,col],1)
        row = pad_col[-1:,:,:]

        pad_row = tf.concat([row, pad_col],0)
        padded.append(pad_row)

    padded = tf.transpose(tf.convert_to_tensor(padded), perm=[3, 1, 2, 0])

    return padded

def kernel_ijk(shape, dtype=None):
  delta = 1 # grid spacing
  kernel = []

  # the kernel is as follows:
  # [0, d/dz, -d/dy]
  # [-d/dz, 0, d/dx]
  # [d/dy, -d/dx, 0]

  ### row 1 ###
  # 0
  kernel.append([[0, 0, 0],
                [0, 0, 0],
                [0, 0, 0]])
  # dA_y/dz
  kernel.append([[-1/(2*delta), 0, 1/(2*delta)],
                [-1/(2*delta), 0, 1/(2*delta)],
                [-1/(2*delta), 0, 1/(2*delta)]])

  # -dA_z/dy
  kernel.append([[1/(2*delta), 1/(2*delta), 1/(2*delta)],
                [0, 0, 0],
                [-1/(2*delta), -1/(2*delta), -1/(2*delta)]])

  ### row 2 ###
  # -dA_x/dz
  kernel.append([[1/(2*delta), 0, -1/(2*delta)],
                [1/(2*delta), 0, -1/(2*delta)],
                [1/(2*delta), 0, -1/(2*delta)]])
  # 0
  kernel.append([[0, 0, 0],
                [0, 0, 0],
                [0, 0, 0]])
  # dA_z/dx
  kernel.append([[-1/(2*delta), 0, 1/(2*delta)],
                [-1/(2*delta), 0, 1/(2*delta)],
                [-1/(2*delta), 0, 1/(2*delta)]])

  ### row 3 ###
  # dA_x/dy
  kernel.append([[-1/(2*delta), -1/(2*delta), -1/(2*delta)],
                [0, 0, 0],
                [1/(2*delta), 1/(2*delta), 1/(2*delta)]])
  # -dA_y/dx
  kernel.append([[1/(2*delta), 0, -1/(2*delta)],
                [1/(2*delta), 0, -1/(2*delta)],
                [1/(2*delta), 0, -1/(2*delta)]])
  # 0
  kernel.append([[0, 0, 0],
                [0, 0, 0],
                [0, 0, 0]])

  kernel = np.asarray(kernel).reshape(shape[2],shape[3],shape[0],shape[1])

  return kernel

def phys_cae(train, test, validation, epochs, batch_size, filenames_train, filenames_test, output_filename,dim):

  # image contains zero padding in place of the ghost cells
  input = layers.Input(shape=(train.shape[1],train.shape[2], train.shape[3]))

  ### encoder ###
  encoder_layer1 = layers.Conv2D(6, 3, 2, activation='relu', padding='same')(input)
  encoder_layer2 = layers.Conv2D(6,  3, 2, activation='relu', padding='same')(encoder_layer1)
  encoder_layer3 = layers.Conv2D(6, 3, 2, activation='relu', padding='same')(encoder_layer2)

  ### decoder ###
  decoder_layer1 = layers.Conv2DTranspose(6, 3, strides=2, activation="relu", padding="same")(encoder_layer3)
  decoder_layer2 = layers.Conv2DTranspose(6, 3, strides=2, activation="relu", padding="same")(decoder_layer1)
  decoder_layer3 = layers.Conv2DTranspose(3, 4, strides = 2, activation=None, padding="same")(decoder_layer2)

  ### phys layer 1: apply periodic boundary conditions ###
  apply_boundary_conditions = PeriodicBoundaryConditions()
  periodic_boundary_conditions = apply_boundary_conditions(decoder_layer3)
  boundary_conditions_layer = layers.Conv2D(3, 3, activation=None, padding="same")(periodic_boundary_conditions)

  ### phys layer 2: apply spatial derivatives as filters ###
  spatial_derivatives_layer = layers.Conv2D(3,3, kernel_initializer=kernel_ijk, activation=None, padding = "same", strides = 1,trainable = False)(boundary_conditions_layer)

  ### phys layer 3:
  curl_layer = layers.Conv2D(3,3, activation=None, padding = "same", strides = 1,trainable = False)(spatial_derivatives_layer)

  # model
  model = keras.Model(input, curl_layer)
  model.compile(optimizer="adam", loss= "mean_squared_error")
  model.summary()

  # fit model and plot loss history
  history = model.fit(train, train, epochs=epochs, batch_size=batch_size, validation_data=(validation, validation))
  plot_history(history,True)

  # model predictions
  train_prediction = model.predict(train)
  test_prediction = model.predict(test)

  # RMSE training and test scores
  RMSE(train_prediction, test_prediction, train, test)

  # plot results
  x, y = generate_scatterplot_indices(test.shape[1])
  generate_plots(test_prediction, filenames_test, train.shape[0]+validation.shape[0], x,y,output_filename,epochs,dim)

In [ ]:
# downsample to 127 x 127, and add zero padding to make 128 x 128
dimensions = 127

# 70% training and validation dataset
train_percent = 0.7
output_filename = "phys_cae_divergence.csv"

# load data
train, test, validation, filenames_train, filenames_test, filenames_validation = load_data_uv(filepath, filenames, train_percent, dimensions)

# pad train and test sets
train = np.pad(train, ((0,0),(1,0),(0,1),(0,0)), 'constant', constant_values=(0,0))
test = np.pad(test, ((0,0),(1,0),(0,1),(0,0)), 'constant', constant_values=(0,0))
validation = np.pad(validation, ((0,0),(1,0),(0,1),(0,0)), 'constant', constant_values=(0,0))

for i in range(0,len(epochs)):
  phys_cae(train, test, validation, epochs[i], batch_size, filenames_train, filenames_test,output_filename,dimensions)

# 35% training and validation dataset
train_percent = 0.35
output_filename = "phys_cae_divergence_5_train.csv"

train, test, validation, filenames_train, filenames_test, filenames_validation = load_data_uv(filepath, filenames, train_percent, dimensions)

# pad train and test sets
train = np.pad(train, ((0,0),(1,0),(0,1),(0,0)), 'constant', constant_values=(0,0))
test = np.pad(test, ((0,0),(1,0),(0,1),(0,0)), 'constant', constant_values=(0,0))
validation = np.pad(validation, ((0,0),(1,0),(0,1),(0,0)), 'constant', constant_values=(0,0))

for i in range(0,len(epochs)):
  phys_cae(train, test, validation, epochs[i], batch_size, filenames_train, filenames_test,output_filename,dimensions)

**Phys CAE without the last layer**

In [ ]:
def phys_cae_no_layer_3(train, test, validation, epochs, batch_size, filenames_train, filenames_test, output_filename,dim):

  # image contains zero padding in place of the ghost cells
  input = layers.Input(shape=(train.shape[1],train.shape[2], train.shape[3]))

  ### encoder ###
  encoder_layer1 = layers.Conv2D(6, 3, 2, activation='relu', padding='same')(input)
  encoder_layer2 = layers.Conv2D(6,  3, 2, activation='relu', padding='same')(encoder_layer1)
  encoder_layer3 = layers.Conv2D(6, 3, 2, activation='relu', padding='same')(encoder_layer2)

  ### decoder ###
  decoder_layer1 = layers.Conv2DTranspose(6, 3, strides=2, activation="relu", padding="same")(encoder_layer3)
  decoder_layer2 = layers.Conv2DTranspose(6, 3, strides=2, activation="relu", padding="same")(decoder_layer1)
  decoder_layer3 = layers.Conv2DTranspose(3, 4, strides = 2, activation=None, padding="same")(decoder_layer2)

  ### phys layer 1: apply periodic boundary conditions ###
  apply_boundary_conditions = PeriodicBoundaryConditions()
  periodic_boundary_conditions = apply_boundary_conditions(decoder_layer3)
  boundary_conditions_layer = layers.Conv2D(3, 3, activation=None, padding="same")(periodic_boundary_conditions)

  ### phys layer 2: apply spatial derivatives as filters ###
  spatial_derivatives_layer = layers.Conv2D(3,3, kernel_initializer=kernel_ijk, activation=None, padding = "same", strides = 1,trainable = False)(boundary_conditions_layer)

  # model
  model = keras.Model(input, spatial_derivatives_layer) #curl_layer)
  model.compile(optimizer="adam", loss="mean_squared_error")
  model.summary()

  # fit model and plot loss history
  history = model.fit(train, train, epochs=epochs, batch_size=batch_size, validation_data=(validation, validation))
  plot_history(history,True)

  # model predictions
  train_prediction = model.predict(train)
  test_prediction = model.predict(test)

  # plot results
  x, y = generate_scatterplot_indices(test.shape[1])
  generate_plots(test_prediction, filenames_test, train.shape[0]+validation.shape[0], x,y,output_filename,epochs,dim)

In [ ]:
# downsample to 127 x 127, and add zero padding to make 128 x 128
dimensions = 127

# 70% training and test dataset
train_percent = 0.7
output_filename = "phys_cae_divergence_no_layer3.csv"

# load data
train, test, validation, filenames_train, filenames_test, filenames_validation = load_data_uv(filepath, filenames, train_percent, dimensions)

# pad train and test sets
train = np.pad(train, ((0,0),(1,0),(0,1),(0,0)), 'constant', constant_values=(0,0))
test = np.pad(test, ((0,0),(1,0),(0,1),(0,0)), 'constant', constant_values=(0,0))
validation = np.pad(validation, ((0,0),(1,0),(0,1),(0,0)), 'constant', constant_values=(0,0))

for i in range(0,len(epochs)):
  phys_cae_no_layer_3(train, test, validation, epochs[i], batch_size, filenames_train, filenames_test,output_filename,dimensions)

# 35% training and validation dataset
train_percent = 0.35
output_filename = "phys_cae_divergence_no_layer3_5_train.csv"

train, test, validation, filenames_train, filenames_test, filenames_validation = load_data_uv(filepath, filenames, train_percent, dimensions)

# pad train and test sets
train = np.pad(train, ((0,0),(1,0),(0,1),(0,0)), 'constant', constant_values=(0,0))
test = np.pad(test, ((0,0),(1,0),(0,1),(0,0)), 'constant', constant_values=(0,0))
validation = np.pad(validation, ((0,0),(1,0),(0,1),(0,0)), 'constant', constant_values=(0,0))

for i in range(0,len(epochs)):
  phys_cae_no_layer_3(train, test, validation, epochs[i], batch_size, filenames_train, filenames_test,output_filename,dimensions)

**Phys CAE without any padding (initial zero padding and BC layer)**

In [ ]:
def phys_cae_no_bc(train, test, validation, epochs, batch_size, filenames_train, filenames_test, output_filename,dim):

  # image contains zero padding in place of the ghost cells
  input = layers.Input(shape=(train.shape[1],train.shape[2], train.shape[3]))

  ### encoder ###
  encoder_layer1 = layers.Conv2D(6, 3, 2, activation='relu', padding='same')(input)
  encoder_layer2 = layers.Conv2D(6,  3, 2, activation='relu', padding='same')(encoder_layer1)
  encoder_layer3 = layers.Conv2D(6, 3, 2, activation='relu', padding='same')(encoder_layer2)

  ### decoder ###
  decoder_layer1 = layers.Conv2DTranspose(6, 3, strides=2, activation="relu", padding="same")(encoder_layer3)
  decoder_layer2 = layers.Conv2DTranspose(6, 3, strides=2, activation="relu", padding="same")(decoder_layer1)
  decoder_layer3 = layers.Conv2DTranspose(3, 4, strides = 2, activation=None, padding="same")(decoder_layer2)

  ### phys layer 1: apply spatial derivatives as filters ###
  spatial_derivatives_layer = layers.Conv2D(3,3, kernel_initializer=kernel_ijk, activation=None, padding = "same", strides = 1,trainable = False)(decoder_layer3) #(boundary_conditions_layer)

  ### phys layer 2:
  curl_layer = layers.Conv2D(3,3, activation=None, padding = "same", strides = 1,trainable = False)(spatial_derivatives_layer)

  # model
  model = keras.Model(input, curl_layer)
  model.compile(optimizer="adam", loss="mean_squared_error")
  model.summary()

  # fit model and plot loss history
  history = model.fit(train, train, epochs=epochs, batch_size=batch_size, validation_data=(validation, validation))
  plot_history(history,True)

  # model predictions
  train_prediction = model.predict(train)
  test_prediction = model.predict(test)

  # plot results
  x, y = generate_scatterplot_indices(test.shape[1])
  generate_plots(test_prediction, filenames_test, train.shape[0]+validation.shape[0], x,y,output_filename,epochs,dim)

In [ ]:
# no zero padding
dimensions = 128

# 70% training and validation dataset
train_percent = 0.7
output_filename = "phys_cae_divergence_no_padding.csv"

train, test, validation, filenames_train, filenames_test, filenames_validation = load_data_uv(filepath, filenames, train_percent, dimensions)

for i in range(0,len(epochs)):
  phys_cae_no_bc(train, test, validation, epochs[i], batch_size, filenames_train, filenames_test,output_filename,dimensions)

# 35% training and validation dataset
train_percent = 0.35
output_filename = "phys_cae_divergence_no_padding_5_train.csv"

train, test, validation, filenames_train, filenames_test, filenames_validation = load_data_uv(filepath, filenames, train_percent, dimensions)

for i in range(0,len(epochs)):
  phys_cae_no_bc(train, test, validation, epochs[i], batch_size, filenames_train, filenames_test,output_filename,dimensions)

**Plot Divergence**

In [ ]:
input_df = pd.read_csv(filepath + "input_divergence.csv", header=None)
input_divergence = input_df.loc[input_df[0] == 1100]

labels = ["PhysCAE, 70% training/validation","PhysCAE, 35% training/validation"]
filenames = ["phys_cae_divergence.csv","phys_cae_divergence_5_train.csv"]
plot_mean_divergence(filenames,input_divergence,sorted(epochs), labels)

labels = ["CAE, padded fields","PhysCAE"]
filenames = ["cae_divergence_padded.csv","phys_cae_divergence.csv"]
plot_mean_divergence(filenames,input_divergence,sorted(epochs), labels)

labels = ["CAE, padded fields, 35% training/validation","PhysCAE, 35% training/validation"]
filenames = ["cae_divergence_padded_5_train.csv","phys_cae_divergence_5_train.csv"]
plot_mean_divergence(filenames,input_divergence,sorted(epochs), labels)

labels = ["CAE","CAE, padded fields"]
filenames = ["cae_divergence.csv","cae_divergence_padded.csv"]
plot_mean_divergence(filenames,input_divergence,sorted(epochs), labels)

labels = ["PhysCAE","PhysCAE, no curl layer","PhysCAE, no bc layer"]
filenames = ["phys_cae_divergence.csv","phys_cae_divergence_no_layer3.csv","phys_cae_divergence_no_padding.csv"]
plot_mean_divergence(filenames,input_divergence,sorted(epochs), labels)

labels = ["CAE","CAE, padded fields","PhysCAE","PhysCAE, no curl layer","PhysCAE, no bc layer"]
filenames = ["cae_divergence.csv","cae_divergence_padded.csv","phys_cae_divergence_5_train.csv","phys_cae_divergence_no_layer3.csv","phys_cae_divergence_no_padding.csv"]
plot_mean_divergence(filenames,input_divergence,sorted(epochs), labels)

labels = ["PhysCAE, 35% training/validation","PhysCAE, no curl layer, 35% training/validation","PhysCAE, no bc layer, 35% training/validation"]
filenames = ["phys_cae_divergence_5_train.csv","phys_cae_divergence_no_layer3_5_train.csv","phys_cae_divergence_no_padding_5_train.csv"]
plot_mean_divergence(filenames,input_divergence,sorted(epochs), labels)

labels = ["CAE, 35% training/validation","CAE, padded fields, 35% training/validation","PhysCAE, 35% training/validation","PhysCAE, no curl layer, 35% training/validation","PhysCAE, no bc layer, 35% training/validation"]
filenames = ["cae_divergence_5_train.csv","cae_divergence_padded_5_train.csv","phys_cae_divergence_5_train.csv","phys_cae_divergence_no_layer3_5_train.csv","phys_cae_divergence_no_padding_5_train.csv"]
plot_mean_divergence(filenames,input_divergence,sorted(epochs), labels)

**Variation on Phys CAE using Three Layers for the Curl Operation**

In [ ]:
def kernel_i(shape, dtype=None):
  delta = 1 # grid spacing
  kernel = []

  # (dA_z/dy - dA_y/dz)i
  for i in range(0,shape[2]):
    # dA_x/dx (set to zero because this is not used)
    kernel.append([[0, 0, 0],
                [0, 0, 0],
                [0, 0, 0]])
    #-dA_y/dz
    kernel.append([[1/(2*delta), 0, -1/(2*delta)],
                [1/(2*delta), 0, -1/(2*delta)],
                [-1/(2*delta), 0, -1/(2*delta)]])
    # dA_z/dy
    kernel.append([[-1/(2*delta), -1/(2*delta), -1/(2*delta)],
                [0, 0, 0],
                [1/(2*delta), 1/(2*delta), 1/(2*delta)]])

  kernel = np.asarray(kernel).reshape(shape[2],shape[3],shape[0],shape[1])
  kernel = kernel.reshape(shape[0],shape[1],shape[2],shape[3])

  return kernel

def kernel_j(shape, dtype=None):
  delta = 1 # grid spacing
  kernel = []

  #(dA_x/dz - dA_z/dx)j
  for i in range(0,shape[2]):
    # dA_x/dz
    kernel.append([[-1/(2*delta), 0, 1/(2*delta)],
                [-1/(2*delta), 0, 1/(2*delta)],
                [-1/(2*delta), 0, 1/(2*delta)]])
    # dA_y/dy (set to zero because this is not used)
    kernel.append([[0, 0, 0],
                [0, 0, 0],
                [0, 0, 0]])
    #-dA_z/dx
    kernel.append([[1/(2*delta), 0, -1/(2*delta)],
                [1/(2*delta), 0, -1/(2*delta)],
                [1/(2*delta), 0, -1/(2*delta)]])

  kernel = np.asarray(kernel).reshape(shape[2],shape[3],shape[0],shape[1])
  kernel = kernel.reshape(shape[0],shape[1],shape[2],shape[3])

  return kernel

def kernel_k(shape, dtype=None):
  delta = 1 # grid spacing
  kernel = []

  #(dA_y/dx - dA_x/dy)k
  for i in range(0,shape[2]):
    # -dA_x/dy
    kernel.append([[1/(2*delta), 1/(2*delta), 1/(2*delta)],
                [0, 0, 0],
                [-1/(2*delta), -1/(2*delta), -1/(2*delta)]])
    # dA_y/dx
    kernel.append([[-1/(2*delta), 0, 1/(2*delta)],
                [-1/(2*delta), 0, 1/(2*delta)],
                [-1/(2*delta), 0, 1/(2*delta)]])
    # dA_z/dz (set to zero because this is not used)
    kernel.append([[0, 0, 0],
                [0, 0, 0],
                [0, 0, 0]])

  kernel = np.asarray(kernel).reshape(shape[2],shape[3],shape[0],shape[1])
  kernel = kernel.reshape(shape[0],shape[1],shape[2],shape[3])

  return kernel

def phys_cae_3_layer_curl(train, test, validation, epochs, batch_size, filenames_train, filenames_test, output_filename,dim):

  # image contains zero padding in place of the ghost cells
  input = layers.Input(shape=(train.shape[1],train.shape[2], train.shape[3]))

  ### encoder ###
  encoder_layer1 = layers.Conv2D(6, 3, 2, activation='relu', padding='same')(input)
  encoder_layer2 = layers.Conv2D(6,  3, 2, activation='relu', padding='same')(encoder_layer1)
  encoder_layer3 = layers.Conv2D(6, 3, 2, activation='relu', padding='same')(encoder_layer2)

  ### decoder ###
  decoder_layer1 = layers.Conv2DTranspose(6, 3, strides=2, activation="relu", padding="same")(encoder_layer3)
  decoder_layer2 = layers.Conv2DTranspose(6, 3, strides=2, activation="relu", padding="same")(decoder_layer1)
  decoder_layer3 = layers.Conv2DTranspose(3, 4, strides = 2, activation=None, padding="same")(decoder_layer2)

  ### phys layer 1: apply periodic boundary conditions ###
  apply_boundary_conditions = PeriodicBoundaryConditions()
  periodic_boundary_conditions = apply_boundary_conditions(decoder_layer3)
  boundary_conditions_layer = layers.Conv2D(3, 3, activation=None, padding="same")(periodic_boundary_conditions)

  ### phys layer 2: apply spatial derivatives as filters ###
  derivatives_i = layers.Conv2D(3,3, kernel_initializer=kernel_i, activation=None, padding = "same", strides = 1,trainable = False)(boundary_conditions_layer)
  derivatives_j = layers.Conv2D(3,3, kernel_initializer=kernel_j, activation=None, padding = "same", strides = 1,trainable = False)(boundary_conditions_layer)
  derivatives_k = layers.Conv2D(3,3, kernel_initializer=kernel_k, activation=None, padding = "same", strides = 1,trainable = False)(boundary_conditions_layer)

  A_ijk = tf.convert_to_tensor([derivatives_i[:,:,:,0],derivatives_j[:,:,:,1],derivatives_k[:,:,:,2]])
  A_ijk = tf.transpose(A_ijk, perm=[1, 2, 3, 0])

  ### phys layer 3:
  curl_layer = layers.Conv2D(3,3, activation=None, padding = "same", strides = 1,trainable = False)(A_ijk)

  # model
  model = keras.Model(input, curl_layer)
  model.compile(optimizer="adam", loss="mean_squared_error")
  model.summary()

  # fit model and plot loss history
  history = model.fit(train, train, epochs=epochs, batch_size=batch_size, validation_data=(validation, validation))
  plot_history(history,True)

  # model predictions
  train_prediction = model.predict(train)
  test_prediction = model.predict(test)

  # plot results
  x, y = generate_scatterplot_indices(test.shape[1])
  generate_plots(test_prediction, filenames_test, train.shape[0]+validation.shape[0], x,y,output_filename,epochs,dim)

In [ ]:
# downsample to 127 x 127, and add zero padding to make 128 x 128
dimensions = 127
train_percent = 0.7
output_filename = "phys_cae_3_layer_curl.csv"

# load data
train, test, validation, filenames_train, filenames_test, filenames_validation = load_data_uv(filepath, filenames, train_percent, dimensions)

# pad train and test sets
train = np.pad(train, ((0,0),(1,0),(0,1),(0,0)), 'constant', constant_values=(0,0))
test = np.pad(test, ((0,0),(1,0),(0,1),(0,0)), 'constant', constant_values=(0,0))
validation = np.pad(validation, ((0,0),(1,0),(0,1),(0,0)), 'constant', constant_values=(0,0))

for i in range(0,len(epochs)):
  phys_cae_3_layer_curl(train, test, validation, epochs[i], batch_size, filenames_train, filenames_test,output_filename,dimensions)

**Test Case for Computing Divergence: compare the results from three variations**

In [ ]:
def compute_div(fields):
  dF_dx, dF_dy, dF_dz = np.gradient(fields)

  print("The sum of du_dx is ", np.sum(dF_dx))
  print("The sum of dv_dy is ", np.sum(dF_dy))
  print("The sum of dw_dz is ", np.sum(dF_dz))
  print("The sum of du_dx + dv/dy + dw/dz is ", np.sum(dF_dx)+np.sum(dF_dy)+np.sum(dF_dz))

  divergence = dF_dx + dF_dy + dF_dz

  return divergence

def compute_divergence_separate_components(fields):
  dF_dx = np.gradient(fields[0])
  dF_dy = np.gradient(fields[1])
  dF_dz = np.gradient(fields[2])

  print("The sum of du_dx is ", np.sum(dF_dx))
  print("The sum of dv_dy is ", np.sum(dF_dy))
  print("The sum of dw_dz is ", np.sum(dF_dz))
  print("The sum of du_dx + dv/dy + dw/dz is ", np.sum(dF_dx)+np.sum(dF_dy)+np.sum(dF_dz))

  divergence = dF_dx + dF_dy + dF_dz

  return divergence

def manually_compute_divergence(fields):
  # based on: https://gist.github.com/GregTJ/24146e6731bae90a5a056c95cb94153d
  shape = 512, 512
  dims = 3

  # compute the jacobian
  partials = tuple(np.gradient(i) for i in fields)
  jacobian = np.stack(partials).reshape(*(j := (dims,2)), *shape)

  # extract divergence from the jacobian using trace
  divergence = np.trace(jacobian)

  return divergence

filename = ["fileu_100.dat","filev_100.dat"]

dimensions = 512 # original dimensions of the field

fieldu = pd.read_table(filepath + filename[0], delimiter=',',header=None).to_numpy()
fieldv = pd.read_table(filepath + filename[1], delimiter=',',header=None).to_numpy()
fieldw = np.zeros((dimensions,dimensions))

fields = [fieldu,fieldv,fieldw]

print("Method 1: Compute divergence as single gradient call")
divergence = compute_div(fields)
sum_field1 = np.sum(divergence)/(512*512)

filenames = ["fileu_100.dat","filev_100.dat"]

print("Method 2: Compute divergence as separate gradient calls")
divergence = compute_divergence_separate_components(fields)
sum_field2 = np.sum(divergence)/(512*512)

print("Method 3: Compute divergence manually")
divergence = manually_compute_divergence(fields)
sum_field3 = np.sum(divergence)/(512*512)

print("Method 1: the sum of the divergence of the vector field across all elements is: ", sum_field1)
print("Method 2: the sum of the divergence of the vector field across all elements is: ", sum_field2)
print("Method 3: the sum of the divergence of the vector field across all elements is: ", sum_field3)

**Test Case for Periodic BCs**

In [ ]:
def bc_padding_test_case():
  filenames = ["fileu_100.dat","filev_100.dat",
             "fileu_200.dat","filev_200.dat",
             "fileu_300.dat","filev_300.dat",
             "fileu_400.dat","filev_400.dat",
             "fileu_500.dat","filev_500.dat"]

  validation_percent = 0.4
  dimensions = 4

  inputs, test1,filenames_train, filenames_test = load_data_uv(filepath, filenames, validation_percent, dimensions)
  ufield = inputs[0,:,:,0]

  # this is the ufield
  #[[ 0.00043109 -0.01509237 -0.03613735 -0.06296321]
  # [ 0.002038   -0.00500754 -0.01705938 -0.03470955]
  # [ 0.01829178  0.01953093  0.01641086  0.00802337]
  # [ 0.0462379   0.05538551  0.06094921  0.06172691]]

  # this is taken from the custom layer
  padded = []
  for i in range(0,inputs.shape[3]):
    col = inputs[slice(None),:,:1,i] # the second index was changed to account for no zero padding
    col = tf.transpose(col, perm=[1, 2, 0])

    crop_input = inputs[slice(None),:,:,i] # the second and third indeces were changed to account for no zero padding
    crop_input = tf.transpose(crop_input, perm=[1, 2, 0])

    pad_col = tf.concat([crop_input,col],1)

    row = pad_col[-1:,:,:]

    pad_row = tf.concat([row, pad_col],0)

    padded.append(pad_row)

  padded = tf.transpose(tf.convert_to_tensor(padded), perm=[3, 1, 2, 0])

  ufield_padded = padded[0,:,:,0]
  print(ufield_padded)

bc_padding_test_case()

# this is the correct, expected padding for ufield
#[[ 0.0462379   0.05538551  0.06094921  0.06172691  0.0462379]
# [ 0.00043109 -0.01509237 -0.03613735 -0.06296321  0.00043109]
# [ 0.002038   -0.00500754 -0.01705938 -0.03470955  0.002038]
# [ 0.01829178  0.01953093  0.01641086  0.00802337  0.01829178]
# [ 0.0462379   0.05538551  0.06094921  0.06172691  0.0462379]]